In [56]:

import pandas as pd
from scipy import sparse
import numpy as np

In [54]:
def get_doc_top_tpcs(doc_distr, topn=10):
        sorted_tpc_indices = np.argsort(doc_distr)[::-1]
        top = sorted_tpc_indices[:topn].tolist()
        return [(k, float(doc_distr[k])) for k in top if doc_distr[k] > 0]

In [2]:
raw_rosie = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/26_jan/df_1.parquet")

In [52]:
path_thetas_en = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/28_jan/poly_rosie_1_30/mallet_output/thetas_EN.npz"
path_thetas_es = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/28_jan/poly_rosie_1_30/mallet_output/thetas_ES.npz"

In [57]:
# en
thetas_en = sparse.load_npz(path_thetas_en).toarray()
df_en = raw_rosie[raw_rosie.lang == "EN"].copy()
df_en["thetas"] = list(thetas_en)
df_en["top_k"] = df_en["thetas"].apply(lambda x: get_doc_top_tpcs(x, topn=10))
df_en["main_topic_thetas"] = df_en["thetas"].apply(lambda x: int(np.argmax(x)))

# es
thetas_es = sparse.load_npz(path_thetas_es).toarray()
df_es = raw_rosie[raw_rosie.lang == "ES"].copy()
df_es["thetas"] = list(thetas_es)
df_es["top_k"] = df_es["thetas"].apply(lambda x: get_doc_top_tpcs(x, topn=10))
df_es["main_topic_thetas"] = df_es["thetas"].apply(lambda x: int(np.argmax(x)))

# concat back
df_rosie = pd.concat([df_en, df_es], ignore_index=True)

In [ ]:
passage_1 = "if stored, warmed and partially consumed milk can continue to be used. It appears safe to continue feeding previously used milk up to one to two hours after it has been prepared. After that, any unused milk should be discarded" #Can frozen and thawed breast milk be refrigerated for more than 24 hours?

passage_2 = "Is it safe for mothers to breastfeed their infant if they have consumed alcohol: Not drinking alcohol is the safest option for breastfeeding mothers." #Is it safe for a breastfeeding mother to consume alcohol every day?

In [59]:
kept = df_rosie[
    ((df_rosie.lang == "ES") | 
    (df_rosie.text.str.contains(passage_1) ) |
    (df_rosie.text.str.contains(passage_2) )) &
    (df_rosie.doc_id != 'EN_810782_119424-17')
].copy()

kept["questions"] = None
kept.loc[kept.text.str.contains(passage_1), "questions"] = [["Can frozen and thawed breast milk be refrigerated for more than 24 hours?"]] * kept.text.str.contains(passage_1).sum()
kept.loc[kept.text.str.contains(passage_2), "questions"] = [["Is it safe for a breastfeeding mother to consume alcohol every day?"]] * kept.text.str.contains(passage_2).sum()

kept.to_parquet("../data/kept_rosie.parquet", index=False)

In [60]:
kept[kept.lang == "EN"]

,id_top,doc_id,id_preproc,document_id,text,len,full_doc,human_labeled,lemmas,lemmas_tr,...,common_id,thetas,main_topic,doc_score,predicted_probability,label,method,top_k,main_topic_thetas,questions
7378,19111,EN_30589_26801-3,19111,26801,Is it safe for mothers to breastfeed their inf...,49,Alcohol Not drinking alcohol is the safest opt...,False,information technology safe mother breastfeed ...,seguro madre amamantar bebé consumir_alcohol b...,...,19112,"[0.0101010101104174, 0.0, 0.0, 0.0, 0.01010101...",11,0.050717,0.747888,1,predicted,"[(15, 0.3939394064134721), (13, 0.252525240653...",15,Is it safe for a breastfeeding mother to consu...
65996,169122,EN_845493_121284-16,169122,121284,"If you’ve frozen and then thawed breast milk, ...",31,"If you’re expressing or pumping breast milk, i...",False,freeze_thaw breast_milk information technology...,congelar_descongelar leche_materno refrigerar ...,...,169122,"[0.0, 0.0, 0.0, 0.0, 0.01851851879446595, 0.0,...",11,0.022983,0.783267,1,predicted,"[(15, 0.4444444510671828), (8, 0.3148148083300...",15,Can frozen and thawed breast milk be refrigera...


In [44]:
import os
import pathlib

path_queries = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/mind/data/mind_runs/rosie/v1/outs_good_model_tpc15/questions_queries")

xlsx_files = [f for f in os.listdir(path_queries) if f.endswith(".xlsx")]
for filename in xlsx_files:
    df_q = pd.read_excel(path_queries / filename)
    
df_q[df_q.question == "Can frozen and thawed breast milk be refrigerated for more than 24 hours?"]

,Unnamed: 0,pass_id,doc_id,passage,full_doc,top_k,question,reason,question_id,queries
